In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import os
import re
from tqdm.notebook import tqdm

In [2]:
dir_raw = '../data/raw/'
dir_processed = '../data/processed'

In [4]:
os.listdir(dir_raw)

['.gitkeep',
 'matches_female.csv',
 'tournaments_male.csv',
 'mens_matches_free.csv',
 'tournaments_female.csv',
 'temp_matches.csv',
 'matches_male.csv']

In [6]:
matches_male = pd.read_csv(dir_raw+'matches_male.csv', index_col = 0)

In [7]:
matches_male.head(10)

,tournament_index,round,players,result
0,0,Quarter-finals,[1] Tayyab Aslam (PAK) bt Farhan Hashmi (PAK),"7-11, 11-9, 11-5, 11-5 (32m)"
1,0,Quarter-finals,[7] Israr Ahmed (PAK) bt [9/16] Waqas Mehboob ...,"11-3, 11-3, 11-8 (23m)"
2,0,Quarter-finals,[4] Amaad Fareed (PAK) bt [5] Farhan Zaman (PAK),"11-8, 11-7, 12-10 (25m)"
3,0,Quarter-finals,[9/16] Nasir Iqbal (PAK) bt [2] Asim Khan (PAK),"11-9, 11-2, 7-11, 11-5 (42m)"
4,0,2nd round,[1] Tayyab Aslam (PAK) bt [9/16] Noor Zaman (PAK),"11-1, 11-9, 11-9 (25m)"
5,0,2nd round,Farhan Hashmi (PAK) bt [8] Zahir Shah (PAK),"11-2, 11-7, 11-7 (23m)"
6,0,2nd round,[7] Israr Ahmed (PAK) bt [9/16] Syed Ali Mujta...,"11-8, 11-5, 11-7 (29m)"
7,0,2nd round,[9/16] Waqas Mehboob (PAK) bt [3] Farhan Mehbo...,"11-8, 11-9, 11-7 (24m)"
8,0,2nd round,[4] Amaad Fareed (PAK) bt [9/16] Muhammad Bila...,"9-11, 11-7, 11-8, 11-2 (26m)"
9,0,2nd round,[5] Farhan Zaman (PAK) bt [9/16] Haris Qasim (...,"11-7, 11-4, 11-8 (26m)"


In [29]:
matches_male.tail(10)

,tournament_index,round,players,result
68023,2760,1st round,[3] Maqsood Ahmed (PAK) bt Maherwan Daruwala (...,"9-3, 9-5, 9-2"
68024,2760,1st round,Frank Donnelly (AUS) bt Tajudin Adi (BRU)\n\t\...,"9-1, 9-7, 9-1"
68025,2760,1st round,Kelvin Smith (AUS) bt Hitoshi Ushiogi (JPN)\n\...,"9-4, 9-1, 9-3"
68026,2760,1st round,Karimullah Khan (PAK) bt Lionel Lau Kuan Yew (...,"9-3, 9-6, 9-2"
68027,2760,1st round,Martin Bodimeade (ENG) bt Seiji Sakamoto (JPN)...,"9-4, 9-3, 10-8"
68028,2760,1st round,Ulf Lagunoff (SWE) bt Gogi Alauddin (PAK)\n\t\...,"9-6, 9-5, 7-9, 9-4"
68029,2760,1st round,Danny Lee (ENG) bt KL Tan (MAS)\n\t\t\t\t\t\t\...,"10-8, 9-1, 9-6"
68030,2760,1st round,Ralf Schuldt (GER) bt James Ng (MAS)\n\t\t\t\t...,"9-4, 9-3, 9-1"
68031,2760,1st round,Aladdin Alouba (EGY) bt Leif Leiner (SWE)\n\t\...,"7-9, 9-2, 9-5, 9-2"
68032,2760,1st round,[2] Qamar Zaman (PAK) bt Teifion Salisbury (WA...,"6-9, 9-1, 6-9, 10-8, 9-6"


In [27]:
matches_male.tournament_index.max()

2760

In [28]:
matches_male[matches_male.tournament_index == 2760]

,tournament_index,round,players,result
67969,2760,Final,[1] Jahangir Khan (PAK) bt Kelvin Smith (AUS)\...,"9-0, 9-0, 9-5"
67970,2760,Third place play-off,[6] Glen Brumby (AUS) bt Sohail Qaiser (PAK)\n...,"9-3, 7-9, 2-9, 9-0, 9-2"
67971,2760,Semi-finals,[1] Jahangir Khan (PAK) bt [6] Glen Brumby (AU...,"9-4, 9-2, 9-5"
67972,2760,Semi-finals,Kelvin Smith (AUS) bt Sohail Qaiser (PAK)\n\t\...,"4-9, 9-3, 9-2, 2-9, 9-5"
67973,2760,Quarter-finals,[1] Jahangir Khan (PAK) bt [8] Magdi Saad (EGY...,"9-0, 9-5, 9-0"
...,...,...,...,...
68028,2760,1st round,Ulf Lagunoff (SWE) bt Gogi Alauddin (PAK)\n\t\...,"9-6, 9-5, 7-9, 9-4"
68029,2760,1st round,Danny Lee (ENG) bt KL Tan (MAS)\n\t\t\t\t\t\t\...,"10-8, 9-1, 9-6"
68030,2760,1st round,Ralf Schuldt (GER) bt James Ng (MAS)\n\t\t\t\t...,"9-4, 9-3, 9-1"
68031,2760,1st round,Aladdin Alouba (EGY) bt Leif Leiner (SWE)\n\t\...,"7-9, 9-2, 9-5, 9-2"


In [15]:
tournaments_male = pd.read_csv(dir_raw+'tournaments_male.csv', index_col = 0)
tournaments_male = tournaments_male.drop(columns=['index'])
tournaments_male.head(5)

,tournament_type,name,location,date,url,matches_downloaded
0,Challenger Tour 10,Pakistan International Championship (M),"Islamabad, Pakistan",11 Dec 2020,http://www.squashinfo.com/events/713-mens-paki...,True
1,Challenger Tour 10,Sihltal Classic (M),"Langnau am Albis, Switzerland",6 Dec 2020,http://www.squashinfo.com/events/8603-mens-sih...,True
2,Challenger Tour 10,BISL International (M),"Quetta, Pakistan",28 Nov 2020,http://www.squashinfo.com/events/8594-mens-bis...,True
3,Challenger Tour 10,Liechtenstein Open (M),"Vaduz, Liechtenstein",22 Nov 2020,http://www.squashinfo.com/events/8587-mens-lie...,True
4,Challenger Tour 5,Austrian Open (M),"Salzburg, Austria",8 Nov 2020,http://www.squashinfo.com/events/8586-mens-aus...,True


In [19]:
tournaments_male['new_index'] = tournaments_male.index[::-1]
tournaments_male.head(5)

,tournament_type,name,location,date,url,matches_downloaded,new_index
0,Challenger Tour 10,Pakistan International Championship (M),"Islamabad, Pakistan",11 Dec 2020,http://www.squashinfo.com/events/713-mens-paki...,True,2807
1,Challenger Tour 10,Sihltal Classic (M),"Langnau am Albis, Switzerland",6 Dec 2020,http://www.squashinfo.com/events/8603-mens-sih...,True,2806
2,Challenger Tour 10,BISL International (M),"Quetta, Pakistan",28 Nov 2020,http://www.squashinfo.com/events/8594-mens-bis...,True,2805
3,Challenger Tour 10,Liechtenstein Open (M),"Vaduz, Liechtenstein",22 Nov 2020,http://www.squashinfo.com/events/8587-mens-lie...,True,2804
4,Challenger Tour 5,Austrian Open (M),"Salzburg, Austria",8 Nov 2020,http://www.squashinfo.com/events/8586-mens-aus...,True,2803


In [30]:
new_matches = matches_male.merge(
    right=tournaments_male['new_index'],
    how='left',
    left_on='tournament_index',
    right_index=True,
)

new_matches.head(10)

,tournament_index,round,players,result,new_index
0,0,Quarter-finals,[1] Tayyab Aslam (PAK) bt Farhan Hashmi (PAK),"7-11, 11-9, 11-5, 11-5 (32m)",2807
1,0,Quarter-finals,[7] Israr Ahmed (PAK) bt [9/16] Waqas Mehboob ...,"11-3, 11-3, 11-8 (23m)",2807
2,0,Quarter-finals,[4] Amaad Fareed (PAK) bt [5] Farhan Zaman (PAK),"11-8, 11-7, 12-10 (25m)",2807
3,0,Quarter-finals,[9/16] Nasir Iqbal (PAK) bt [2] Asim Khan (PAK),"11-9, 11-2, 7-11, 11-5 (42m)",2807
4,0,2nd round,[1] Tayyab Aslam (PAK) bt [9/16] Noor Zaman (PAK),"11-1, 11-9, 11-9 (25m)",2807
5,0,2nd round,Farhan Hashmi (PAK) bt [8] Zahir Shah (PAK),"11-2, 11-7, 11-7 (23m)",2807
6,0,2nd round,[7] Israr Ahmed (PAK) bt [9/16] Syed Ali Mujta...,"11-8, 11-5, 11-7 (29m)",2807
7,0,2nd round,[9/16] Waqas Mehboob (PAK) bt [3] Farhan Mehbo...,"11-8, 11-9, 11-7 (24m)",2807
8,0,2nd round,[4] Amaad Fareed (PAK) bt [9/16] Muhammad Bila...,"9-11, 11-7, 11-8, 11-2 (26m)",2807
9,0,2nd round,[5] Farhan Zaman (PAK) bt [9/16] Haris Qasim (...,"11-7, 11-4, 11-8 (26m)",2807


In [ ]:
new_matches